In [1]:
import pandas as pd

from SMILESX import main, inference, utils
%load_ext autoreload
%aimport SMILESX
%autoreload 1

%matplotlib inline

RDKit WARNING: [09:23:04] Enabling RDKit 2019.09.2 jupyter extensions


#### **Read data file**

In [2]:
validation_data_dir = "./validation_data/"

In [3]:
extension = '.csv'

In [4]:
data_name = 'FreeSolv' # FreeSolv, ESOL, Lipophilicity
prop_tag = ''

In [5]:
if data_name == 'FreeSolv':
    data_filename = 'FreeSolv_SAMPL'
    prop_tag = 'expt'
elif data_name == 'ESOL':
    data_filename = 'ESOL_delaney-processed'
    prop_tag = 'measured log solubility in mols per litre'
elif data_name == 'Lipophilicity':
    data_filename = 'Lipophilicity'
    prop_tag = 'exp'
else:
    data_filename = data_name
    prop_tag = prop_tag

In [6]:
sol_data = pd.read_csv(validation_data_dir+data_filename+extension)

In [7]:
sol_data.head(3)

,Unnamed: 0,iupac,smiles,expt,calc
0,0,"4-methoxy-N,N-dimethyl-benzamide",COc1ccc(C(=O)N(C)C)cc1,-11.01,-9.625
1,1,methanesulfonyl chloride,CS(=O)(=O)Cl,-4.87,-6.219
2,2,3-methylbut-1-ene,C=CC(C)C,1.83,2.452


#### **Observation**
* The column containing the SMILES must be named 'smiles' 

#### **Extract relevant data**

In [8]:
sol_data = sol_data[['smiles',prop_tag]]

In [9]:
sol_data.head()

,smiles,expt
0,COc1ccc(C(=O)N(C)C)cc1,-11.01
1,CS(=O)(=O)Cl,-4.87
2,C=CC(C)C,1.83
3,CCc1cnccn1,-5.45
4,CCCCCCCO,-4.21


In [10]:
sol_data.shape

(642, 2)

#### **SMILES check from RDKit**

In [11]:
sol_data, bad_smiles_list = utils.check_smiles(sol_data)

In [12]:
sol_data.shape

(642, 2)

In [13]:
sol_data.iloc[:,1].values.shape

(642,)

#### Hyperparameters optimization with GPyOpt (Bayesian optimization)

In [20]:
dhyp_range = [int(2**itn) for itn in range(1,11)] # 

if data_name != 'Lipophilicity':
    bounds = [
        {'name': 'lstmunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'denseunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'embedding', 'type': 'discrete', 'domain': dhyp_range}
    ]
else:
    bounds = [
        {'name': 'lstmunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'denseunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'embedding', 'type': 'discrete', 'domain': dhyp_range}
    ]

In [24]:
main.Main(data=sol_data, 
          data_name=data_name, 
          data_units='', 
          bayopt_bounds=bounds, 
          k_fold_number = 3, 
          augmentation = True, 
          outdir = "./data/", 
          bayopt_n_rounds = 5, 
          bayopt_on = True, 
          lstmunits_ref = 128, 
          denseunits_ref = 16, 
          embedding_ref = 32, 
          seed_ref = None, 
          n_gpus = 1,
          gpus_list = None, 
          gpus_debug = False,
          batchsize_pergpu = 64,  
          patience = 25, 
          n_epochs = 400, 
          lr_schedule = 'decay')

1 Physical GPUs, 1 Logical GPUs detected and configured.
1 GPU device(s) will be used.

***Data augmentation to True***

***SMILES_X starts...***


******
***Fold #0 initiated...***
******
***Splitting and standardization of the dataset.***

Scaler: RobustScaler(copy=True, quantile_range=(5.0, 95.0), with_centering=True,
             with_scaling=True)
Train/valid/test splits: 0.67/0.17/0.17


Enumerated SMILES:
	Training set: 3538
	Validation set: 892
	Test set: 925

***Tokenization of SMILES.***

Examples of tokenized SMILES from a training set:
[[' ', 'C', 'S', '(', '=', 'O', ')', '(', '=', 'O', ')', 'Cl', ' '], [' ', 'S', '(', '=', 'O', ')', '(', '=', 'O', ')', '(', 'Cl', ')', 'C', ' '], [' ', 'O', '=', 'S', '(', '=', 'O', ')', '(', 'Cl', ')', 'C', ' '], [' ', 'O', '=', 'S', '(', 'Cl', ')', '(', 'C', ')', '=', 'O', ' '], [' ', 'Cl', 'S', '(', 'C', ')', '(', '=', 'O', ')', '=', 'O', ' ']]

Number of tokens only present in a training set: 31

Number of tokens only present in a valida

In [13]:
pred_from_ens = inference.Inference(data_name=data_name, 
                                    smiles_list = ['CC','CCC','C=O','ABC','DEF'], 
                                    data_units = '',
                                    k_fold_number = 3,
                                    augmentation = True, 
                                    outdir = "./data/")

***SMILES_X for inference starts...***


***Checking the SMILES list for inference***

***Data augmentation.***

Enumerated SMILES: 5

***Tokenization of SMILES.***

Full vocabulary: ['pad', 'unk', 'Cl', '[N+]', '\\', ')', '2', '-', 'c', '1', 'P', 'n', '=', ' ', '[C@@]', 'S', '3', 'Br', '#', '/', '[O-]', '[C@@H]', '[C@H]', '[nH]', '5', '4', 'C', 's', '(', 'N', 'F', '[S+2]', 'I', '[C@]', 'O']
Of size: 35

Maximum length of tokenized SMILES: 51 tokens

***Inference of SMILES property done.***


In [14]:
pred_from_ens

,SMILES,ens_pred_mean,ens_pred_sd
0,CC,0.406713,0.0213377
1,CCC,0.43053,0.020964
2,C=O,0.00962523,0.0223681
